<a href="https://colab.research.google.com/github/sardaashutosh04/sarvam-ai-assignment/blob/main/SarvamAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install langchain-community neo4j langchain langchain-openai --quiet #beautifulsoup4 PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.1 MB/s eta 0:00:00
     ━━━

## Extract Data

In [ ]:
import os
import io
import json

import requests
import pickle
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup
from google.colab import files

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = files.upload()

In [ ]:
with open('/content/rbi_notification_toc_2023_1991_20230930_212821.json', 'rb') as f:
    data = json.load(f)
os.mkdir('/content/data')
for y, c in data.items():
    with open(f'/content/data/{y}.html', 'w') as f:
        f.write(c)


In [ ]:
def is_date(date):
    valid_months_short = ['Jan', 'Feb', 'Mar', 'Apr',
                          'May', 'Jun', 'Jul', 'Aug',
                          'Sep', 'Oct', 'Nov', 'Dec']
    valid_months_long = ['January', 'Febuary', 'March', 'April',
                         'May', 'June', 'July', 'August',
                         'September', 'October', 'November', 'December']
    date = date.strip()

    try:
        date, year = date.split(',')
        if int(year) < 1991 or int(year) > 2023:
            return False

        mon, date = date.split(' ')
        if mon not in valid_months_short:
            if mon not in valid_months_long:
                return False

        if int(date) > 31 or int(date) < 1:
            return False
    except:
        return False

    return True


In [ ]:
def extract_from_pdf(pdf_url):
    if pdf_url is None:
        return ''
    if not pdf_url.startswith('https://rbidocs.rbi.org.in'):
        if pdf_url:
            print(f'Incorrect URL: {pdf_url}')
        return ''
    r = requests.get(pdf_url)
    f = io.BytesIO(r.content)

    reader = PdfReader(f)
    info = ''
    for page in range(len(reader.pages)):
        info += reader.pages[page].extract_text()
    return info

In [ ]:
def extract_notification_from_table(table):
    try:
        pdf_url = table.find('a')['href']
    except:
        pdf_url = ''

    try:
        title = table.find('b').text
    except:
        title = ''

    date_flag = False
    info = ''
    date = ''
    for p in table.findAll('p'):
        if not date_flag:
            if is_date(p.text):
                date = p.text
    try:
        info = extract_from_pdf(pdf_url)
    except Exception as e:
        print(f'ERROR: extracting info from {pdf_url} failed due to exception {e}. \nNotification: {title}')

    notification = {
        'title': title,
        'date': date,
        'info': info,
        'source': pdf_url
    }
    return notification

In [ ]:
done_files = ['1991.html', '1992.html', '1993.html', '1994.html', '1995.html', '1996.html',
              '1997.html', '1998.html', '1999.html', '2000.html', '2001.html', '2002.html',
              '2003.html', '2004.html', '2005.html', '2006.html', '2007.html', '2008.html',
              '2009.html', '2010.html', '2011.html', '2012.html', '2013.html', '2014.html',
              '2015.html', '2016.html', '2017.html', '2018.html', '2019.html', '2020.html',
              '2021.html', '2022.html', '2023.html']


In [ ]:
%%time
directory = '/content/data'
for filename in os.listdir(directory):
    if filename.endswith('.html') and filename not in done_files:
        print(f'Extracting info from {filename}...')
        notifications = []
        HTMLFileToBeOpened = open(os.path.join(directory, filename), "r")
        contents = HTMLFileToBeOpened.read()
        beautifulSoupText = BeautifulSoup(contents, 'html.parser')
        date = None
        contents = beautifulSoupText.find('tbody').contents
        for content in contents:
            if content.find('b'):
                if is_date(content.find('b').text):
                    date = content.find('b').text
                    continue
                else:
                    continue
            elif content.findAll('a'):
                for c in content.findAll('a'):
                    title = None
                    info = None
                    pdf_url = None
                    if c.text:
                        title = c.text
                    else:
                        pdf_url = c['href']
                        try:
                            info = extract_from_pdf(pdf_url)
                        except Exception as e:
                            print(f'ERROR: extracting info from {pdf_url} failed due to exception {e}. \nNotification: {title}')
                            info = ''
                notifications.append({
                    'title': title,
                    'date': date,
                    'info': info,
                    'source': pdf_url
                })
        output_filename = f'/content/rbi_notifications_{filename.split(".")[0]}_with_info.json'
        with open(output_filename, 'w') as f:
            json.dump(notifications, f)
        # files.download(output_filename)
        print(f'Extraction of notifications from file {filename} complete!')


In [ ]:
with open('/content/filenames_notification_not_in_toc.pkl', 'rb') as f:
    data = pickle.load(f)
data = data[0]
notifications = []

In [ ]:
i = 1 + 5000 + 1000 + 1000 + 1000 + 1000
for url in list(data.keys())[i-1:]:
    try:
        response = requests.get(url)
        contents = response.content
        beautifulSoupText = BeautifulSoup(contents, 'html.parser')
        table = beautifulSoupText.find('table')
        notifications.append(extract_notification_from_table(table))
    except Exception as e:
        print(f'ERROR: extracting info from {url} failed due to exception {e}.')
        continue

    if i%1000 == 0:
        print(f"{i-999}-{i} notifications fetched!")
        output_filename = f'/content/rbi_notifications_not_in_toc_{i-999}_{i}_with_info.json'
        with open(output_filename, 'w') as f:
            json.dump(notifications, f)
            notifications = []
    i += 1

output_filename = '/content/rbi_notifications_not_in_toc_10001_10566_with_info.json'
with open(output_filename, 'w') as f:
    json.dump(notifications, f)

## Creating Knowledge Graph

In [4]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_core.embeddings import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

import tensorflow_hub as hub
import tensorflow as tf

import textwrap
import gc

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open('/content/drive/MyDrive/SarvamAi-RAG/userdata.json', 'r') as f:
    userdata = json.load(f)

NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
NEO4J_DB = 'neo4j'
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


# Global constants
VECTOR_INDEX_NAME = 'notification_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'


In [6]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DB
)

### Adding Nodes

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)


def split_notifications_data_from_file(notification):
    chunks_with_metadata = []
    item_text = notification['info']
    if item_text is None:
        item_text = ''
    item_text_chunks = text_splitter.split_text(item_text)
    chunk_seq_id = 0
    title = notification['title']
    date = notification['date']
    source = notification['source']
    for chunk in item_text_chunks:
        # finally, construct a record with metadata and the chunk text
        chunks_with_metadata.append({
            'text': chunk,
            'chunkSeqId': chunk_seq_id,
            # constructed metadata...
            'chunkId': f'{title}-chunk{chunk_seq_id:04d}',
            # metadata from file...
            'title': title,
            'date': date,
            'source': source,
        })
        chunk_seq_id += 1
    return chunks_with_metadata

In [16]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET
        mergedChunk.title = $chunkParam.title,
        mergedChunk.date = $chunkParam.date,
        mergedChunk.source = $chunkParam.source,
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId,
        mergedChunk.text = $chunkParam.text
RETURN mergedChunk
"""

distinct_chunk_constraint_query = """
CREATE CONSTRAINT unique_chunk IF NOT EXISTS
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
"""

create_vector_index_query = f"""
         CREATE VECTOR INDEX `{VECTOR_INDEX_NAME}` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding)
          OPTIONS {{ indexConfig: {{
            `vector.dimensions`: 512,
            `vector.similarity_function`: 'cosine'
         }}}}
"""


In [10]:
done_files = ['rbi_notifications_2008_with_info.json', 'rbi_notifications_2000_with_info.json',
              'rbi_notifications_2015_with_info.json', 'rbi_notifications_2012_with_info.json',
              'rbi_notifications_2007_with_info.json', 'rbi_notifications_1991_with_info.json',
              'rbi_notifications_1992_with_info.json', 'rbi_notifications_1999_with_info.json',
              'rbi_notifications_1995_with_info.json', 'rbi_notifications_1998_with_info.json',
              'rbi_notifications_1994_with_info.json', 'rbi_notifications_2022_with_info.json',
              'rbi_notifications_2023_with_info.json', 'rbi_notifications_1993_with_info.json',
              'rbi_notifications_2014_with_info.json', 'rbi_notifications_2021_with_info.json',
              'rbi_notifications_1997_with_info.json', 'rbi_notifications_1996_with_info.json',
              'rbi_notifications_2020_with_info.json', 'rbi_notifications_2009_with_info.json',
              'rbi_notifications_2001_with_info.json', 'rbi_notifications_2017_with_info.json',
              'rbi_notifications_2010_with_info.json', 'rbi_notifications_2016_with_info.json',
              'rbi_notifications_2019_with_info.json', 'rbi_notifications_2003_with_info.json',
              'rbi_notifications_2004_with_info.json', 'rbi_notifications_2011_with_info.json',
              'rbi_notifications_2002_with_info.json', 'rbi_notifications_2005_with_info.json',
              'rbi_notifications_2006_with_info.json', 'rbi_notifications_2013_with_info.json',
              'rbi_notifications_2018_with_info.json', 'rbi_notifications_not_in_toc_1_1000_with_info.json',
              'rbi_notifications_not_in_toc_3001_4000_with_info.json', 'rbi_notifications_not_in_toc_1001_2000_with_info.json',
              'rbi_notifications_not_in_toc_2001_3000_with_info.json', 'rbi_notifications_not_in_toc_8001_9000_with_info.json',
              'rbi_notifications_not_in_toc_4001_5000_with_info.json', 'rbi_notifications_not_in_toc_5001_6000_with_info.json',
              'rbi_notifications_not_in_toc_6001_7000_with_info.json', 'rbi_notifications_not_in_toc_7001_8000_with_info.json',
              'rbi_notifications_not_in_toc_9001_10000_with_info.json', 'rbi_notifications_not_in_toc_10001_10566_with_info.json']

In [ ]:
DATA_PATH = '/content/drive/MyDrive/SarvamAi-RAG/data'

In [ ]:
for notifications_file in os.listdir(DATA_PATH):
    if notifications_file not in done_files:
        with open(os.path.join(DATA_PATH, notifications_file), 'r') as f:
            notifications = json.load(f)

        kg.query(distinct_chunk_constraint_query)
        for notification in notifications:
            notification_chunks = split_notifications_data_from_file(notification)
            for chunk in notification_chunks:
                kg.query(merge_chunk_node_query,
                params={
                    'chunkParam': chunk
                })
        print(f'Chunk Nodes added to knowledge graph from file {notifications_file}')

In [17]:
kg.query(create_vector_index_query)

[]

In [ ]:
kg.query("SHOW INDEXES")

### Text Embedding

In [43]:
class CustomTensorflowHubEmbeddings(Embeddings):

    def __init__(self):
        self.model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    def embed_documents(self, texts):
        if texts:
            embeddings = []
            for embedding in self.model(texts):
                embeddings.append(embedding.numpy())
            return embeddings
        else:
            return [[]]

    def embed_query(self, text):
        return self.model([text])[0].numpy()

embeddings_model = CustomTensorflowHubEmbeddings()

In [8]:
chunk_nodes = kg.query("""
    MATCH (chunk: Chunk) WHERE chunk.textEmbedding IS NULL
    RETURN chunk.chunkId, chunk.text""")

create_text_embeddings_query = """
    MATCH (chunk:Chunk) WHERE chunk.chunkId = $id
    WITH chunk, $embedding AS vector
    CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
    """

for chunk in chunk_nodes:
    kg.query(create_text_embeddings_query,
             params={'id': chunk['chunk.chunkId'],
                     'embedding': embeddings_model.embed_query(chunk['chunk.text'])})

In [8]:
kg.query("""
    MATCH (chunk: Chunk) WHERE chunk.textEmbedding IS NULL
    RETURN count(chunk)""")

[{'count(chunk)': 0}]

In [10]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:
Chunk {chunkId: STRING, title: STRING, date: STRING, source: STRING, chunkSeqId: INTEGER, text: STRING, textEmbedding: LIST}
Relationship properties are the following:

The relationships are the following:



#### Query Similarity Search

In [48]:
def neo4j_vector_search(question):
    """Search for similar nodes using the Neo4j vector index"""
    question_embedding = embeddings_model.embed_query(question)

    vector_search_query = """
    WITH $question_embedding AS question_vector
    CALL db.index.vector.queryNodes($index_name, $top_k, question_vector) yield node, score
    RETURN score, node.title as title, node.date as date, node.text AS text
    """
    similar = kg.query(vector_search_query,
                        params={
                        'question_embedding': question_embedding,
                        'index_name':VECTOR_INDEX_NAME,
                        'top_k': 10})
    return similar

In [49]:
neo4j_vector_search("Policy changes in 2000?")[0]

{'score': 0.6575847864151001,
 'title': 'Foreign Exchange Management (Borrowing or Lending in Foreign Exchange) (Amendment) Regulations, 2000',
 'date': '',
 'text': 'Foreign Exchange Management (Borrowing or Lending in Foreign Exchange) \n(Amendment) Regulations, 2000 \n \nReserve Bank of India \n(Exchange Control Department) \nCentral Office \nMumbai 400 001 \n Notification No.FEMA 26 /2000-RB \ndated 14\nth August, 2000 \n Foreign Exchange Management (Borrowing or Lending in Foreign Exchange) (Amendment) Regulations, 2000 \n \n In exercise of the powers c onferred by clause (d) of sub-se ction (3) of section 6, and sub-\nsection (2) of Section 47 of the Forei gn Exchange Management Act, 1999 (42 of 1999), and in \npartial modification of its notif ication No.FEMA 3/2000-RB dated 3\nrd May 2000, the Reserve Bank \nof India makes the following Regulations  to amend the Foreign Exchange Management \n(Borrowing or Lending in Foreign Exchange) Regulations 2000, namely :- \n \n2. Short ti

### Adding Relationships

#### Adding Notification Node

In [64]:
query = cypher = """
    MATCH (c:Chunk)
    WITH c
    RETURN c { .title, .date, .source } as notifyMeta
"""

notification_metadata_list = kg.query(query)

In [65]:
notification_metadata_list[0]

{'notifyMeta': {'title': 'StCBs/DCCBs - KYC Norms/AML Standards/Combating Financing of Terrorism/Obligation of Banks under PMLA, 2002',
  'source': 'https://rbidocs.rbi.org.in/rdocs/notification/PDFs/RFBC89D250610.pdf',
  'date': 'June 25, 2010'}}

In [ ]:
distinct_notification_constraint_query = """
CREATE CONSTRAINT unique_notification IF NOT EXISTS
    FOR (n:Notification) REQUIRE n.id IS UNIQUE
"""

kg.query(distinct_notification_constraint_query)
kg.query("SHOW INDEXES")

In [ ]:
cypher = """
    MERGE (n:Notification {title: $notificationMetadata.title })
      ON CREATE
        SET n.date = $notificationMetadata.date
        SET n.source = $notificationMetadata.source
        SET n.id = $id
"""

i = 1
for notification_metadata in notification_metadata_list[i-1:]:
    if notification_metadata['notifyMeta']['title'] is None:
        notification_metadata['notifyMeta']['title'] = ''
    id = f"{notification_metadata['notifyMeta']['title']}-{notification_metadata['notifyMeta']['date']}-{notification_metadata['notifyMeta']['source']}"
    kg.query(cypher,
             params={'notificationMetadata': notification_metadata['notifyMeta'],
                     'id': id})

    if i%1000 == 0:
        print(f"{i} Notification Nodes created")
    i += 1

In [96]:
notitfication_nodes = kg.query("""
    MATCH (n:Notification)
    RETURN n""")

In [97]:
len(notitfication_nodes)

4434

In [98]:
notitfication_nodes[0]

{'n': {'date': '',
  'source': 'https://rbidocs.rbi.org.in/rdocs/notification/PDFs/60ME300611F.pdf',
  'id': 'Master Circular on Rupee / Foreign Currency Export Credit & Customer Service To Exporters--https://rbidocs.rbi.org.in/rdocs/notification/PDFs/60ME300611F.pdf',
  'title': 'Master Circular on Rupee / Foreign Currency Export Credit & Customer Service To Exporters'}}

### Chaining Chunks

In [99]:
cypher = """
  MATCH (from_same_section:Chunk)
  WHERE from_same_section.title = $title
    AND from_same_section.source = $source
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqId ASC
  WITH collect(from_same_section) as chunk_list
    CALL apoc.nodes.link(
        chunk_list,
        "NEXT",
        {avoidDuplicates: true}
    )
  RETURN size(chunk_list)
"""

i = 1
for notification_node in notitfication_nodes[i-1:]:
    kg.query(cypher, params={'title': notification_node['n']['title'],
                             'source': notification_node['n']['source']})
    if i %1000 == 0:
        print(f'Added relations for {i} notifications')
    i += 1

Added relations for 1000 notifications
Added relations for 2000 notifications
Added relations for 3000 notifications
Added relations for 4000 notifications


In [102]:
kg.query("""
    MATCH (n)-[r]-()
    RETURN count(r)""")

[{'count(r)': 54922}]

### Relation from Notification to Chunk

In [103]:
cypher = """
  MATCH (c:Chunk), (n:Notification)
    WHERE c.title = n.title
        AND c.source = n.source
  MERGE (c)-[newRelationship:PART_OF]->(n)
  RETURN count(newRelationship)
"""

kg.query(cypher)

[{'count(newRelationship)': 31895}]

In [116]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:
Chunk {chunkId: STRING, title: STRING, date: STRING, source: STRING, chunkSeqId: INTEGER, text: STRING, textEmbedding: LIST},Notification {title: STRING, date: STRING, source: STRING, id: STRING}
Relationship properties are the following:

The relationships are the following:
(:Chunk)-[:NEXT]->(:Chunk),(:Chunk)-[:PART_OF]->(:Notification)


## LangChain RAG Workflow

In [119]:
retrieval_query_window = """
MATCH window=
    (:Chunk)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Chunk)
WITH node, score, window as longestWindow
  ORDER BY length(window) DESC LIMIT 3
WITH nodes(longestWindow) as chunkList, node, score
  UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node, score
RETURN apoc.text.join(textList, " \n ") as text,
    score,
    node {.title, .date, .source} AS metadata
"""

In [120]:
vector_store_window = Neo4jVector.from_existing_index(
    embedding=CustomTensorflowHubEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DB,
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=retrieval_query_window,
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain_window = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
    chain_type="stuff",
    retriever=retriever_window
)

In [123]:
def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain_window({"question": question},
        return_only_outputs=True,)
    print(textwrap.fill(response['answer'], 60))

In [138]:
prettychain("What were the major policies by RBI in 2000?")

The major policies by RBI in 2000 included the empowerment
to compound contraventions relating to Section 7, 8, and 9
of FEMA 1999. Additionally, RBI was empowered to compound
all contraventions of FEMA 1999 except Section 3(a) with a
view to providing comfort to individuals and the corporate
community by minimizing transaction costs while taking a
severe view of willful, malafide, and fraudulent
transactions.
